In [1]:
from geexhp.model import datasetup as dset

import tensorflow as tf
import pandas as pd
import numpy as np

import os

from tqdm import tqdm

from astropy.constants import R_earth

2025-01-13 16:34:46.449133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736796886.467911 2778956 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736796886.473334 2778956 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 16:34:46.494809: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Auxiliary Functions

In [2]:
# Helper functions to create tf.train.Features
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _float_feature(value):
    """Returns a float_list from a float / list of floats."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _float_feature_list(value):
    """Returns a float_list from a float / list of floats."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [3]:
# Function to serialize a row into tf.train.Example
def serialize_sample(row):
    feature = {
        'NOISY_ALBEDO_B-NIR' : _float_feature_list(row['NOISY_ALBEDO_B-NIR']),
        'NOISY_ALBEDO_B-UV' : _float_feature_list(row['NOISY_ALBEDO_B-UV']),
        'NOISY_ALBEDO_B-Vis' : _float_feature_list(row['NOISY_ALBEDO_B-Vis']),
        'NOISY_ALBEDO_SS-NIR' : _float_feature_list(row['NOISY_ALBEDO_SS-NIR']),
        'NOISY_ALBEDO_SS-UV' : _float_feature_list(row['NOISY_ALBEDO_SS-UV']),
        'NOISY_ALBEDO_SS-Vis' : _float_feature_list(row['NOISY_ALBEDO_SS-Vis']),

        'NOISY_ALBEDO_B-NIR_norm' : _float_feature_list(row['NOISY_ALBEDO_B-NIR_norm']),
        'NOISY_ALBEDO_B-UV_norm' : _float_feature_list(row['NOISY_ALBEDO_B-UV_norm']),
        'NOISY_ALBEDO_B-Vis_norm' : _float_feature_list(row['NOISY_ALBEDO_B-Vis_norm']),
        'NOISY_ALBEDO_SS-NIR_norm' : _float_feature_list(row['NOISY_ALBEDO_SS-NIR_norm']),
        'NOISY_ALBEDO_SS-UV_norm' : _float_feature_list(row['NOISY_ALBEDO_SS-UV_norm']),
        'NOISY_ALBEDO_SS-Vis_norm' : _float_feature_list(row['NOISY_ALBEDO_SS-Vis_norm']),

        'radius' : _float_feature(row['radius']),
        'log_gravity' : _float_feature(row['log_gravity']),
        'log_temperature' : _float_feature(row['log_temperature']),
        'log_pressure' : _float_feature(row['log_pressure']),

        'OBJECT-DIAMETER' : _float_feature(row['OBJECT-DIAMETER']),
        'OBJECT-GRAVITY' : _float_feature(row['OBJECT-GRAVITY']),
        'ATMOSPHERE-TEMPERATURE' : _float_feature(row['ATMOSPHERE-TEMPERATURE']),
        'ATMOSPHERE-PRESSURE' : _float_feature(row['ATMOSPHERE-PRESSURE']),
        
        'Earth_type' : _bytes_feature(row['Earth_type']),
        # 'C2H6' : _float_feature(row['C2H6']),
        # 'CH4' : _float_feature(row['CH4']),
        # 'CO' : _float_feature(row['CO']),
        # 'CO2' : _float_feature(row['CO2']),
        # 'H2O' : _float_feature(row['H2O']),
        # 'N2' : _float_feature(row['N2']),
        # 'N2O' : _float_feature(row['N2O']),
        # 'O2' : _float_feature(row['O2']),
        # 'O3' : _float_feature(row['O3']),

        'log_C2H6' : _float_feature(row['log_C2H6']),
        'log_CH4' : _float_feature(row['log_CH4']),
        'log_CO' : _float_feature(row['log_CO']),
        'log_CO2' : _float_feature(row['log_CO2']),
        'log_H2O' : _float_feature(row['log_H2O']),
        'log_N2' : _float_feature(row['log_N2']),
        'log_N2O' : _float_feature(row['log_N2O']),
        'log_O2' : _float_feature(row['log_O2']),
        'log_O3' : _float_feature(row['log_O3']),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [4]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [5]:
def convert_to_earth_radius(data):
    return data/(2/1000 * R_earth.value)

In [6]:
def log_data(data):
    return np.log(data)

# Process

In [7]:
columns_of_interest = [
    'NOISY_ALBEDO_B-NIR',
    'NOISY_ALBEDO_B-UV',
    'NOISY_ALBEDO_B-Vis',
    'NOISY_ALBEDO_SS-NIR',
    'NOISY_ALBEDO_SS-UV',
    'NOISY_ALBEDO_SS-Vis',
    'OBJECT-DIAMETER',
    'OBJECT-GRAVITY',
    'ATMOSPHERE-TEMPERATURE',
    'ATMOSPHERE-PRESSURE',
    'Earth_type'
]

molecules = [
    'C2H6',
    'CH4',
    'CO',
    'CO2',
    'H2O',
    'N2',
    'N2O',
    'O2',
    'O3'
]

spectra = [
    'NOISY_ALBEDO_B-NIR',
    'NOISY_ALBEDO_B-UV',
    'NOISY_ALBEDO_B-Vis',
    'NOISY_ALBEDO_SS-NIR',
    'NOISY_ALBEDO_SS-UV',
    'NOISY_ALBEDO_SS-Vis'
]

In [8]:
root_folder = '/home/cleber/Documentos/Sarah/geexhp-main/TFRecord_Tests/parallel'

folders = os.listdir(root_folder)

# Count the total number of '.parquet' files
file_count = sum(
    len([file for file in files if file.endswith('.parquet')])
    for _, _, files in os.walk(root_folder)
)

In [9]:
with tqdm(
    total=file_count,
    desc="🌍 Progress",
    dynamic_ncols=True,
    colour='cyan',
    bar_format="{desc}: |{bar:30}| {percentage:3.0f}% ({n_fmt}/{total_fmt} files) ⏳ [{elapsed} elapsed]"
) as pbar:
    
    for folder in folders:
        folder_path = os.path.join(root_folder, folder)

        if not os.path.isdir(folder_path):
            continue  # Skip stuff that is not a folder

        files = os.listdir(folder_path)

        for file in files:
            # Some code
            file_path = os.path.join(folder_path, file)

            if not file.endswith(".parquet"):
                continue  # Skip non-parquet files

            earth_type = file.split("_")[0]
            original_parquet_range = file.split("_")[1].split(".")[0]
            
            df = pd.read_parquet(file_path)
            df["Earth_type"] = earth_type

            noise_columns = [col for col in df.columns if "NOISE_" in col]
            mask = ~df[noise_columns].map(lambda x: any(value > 10 for value in x)).any(axis=1)
            df = df[mask]

            filtered_df = df.copy()
            filtered_df = filtered_df[columns_of_interest]

            df = dset.extract_abundances(df)

            # Convert features to better values.
            filtered_df['radius'] = df['OBJECT-DIAMETER'].apply(convert_to_earth_radius)
            filtered_df['log_gravity'] = df['OBJECT-GRAVITY'].apply(log_data)
            filtered_df['log_temperature'] = df['ATMOSPHERE-TEMPERATURE'].apply(log_data)
            filtered_df['log_pressure'] = df['ATMOSPHERE-PRESSURE'].apply(log_data)
            
            # Apply normalization to the array columns.
            for spectrum in spectra:
                filtered_df[spectrum+'_norm'] = filtered_df[spectrum].apply(NormalizeData)

            # Get all the molecules abundances.
            for molecule in molecules:
                if molecule in df.columns:
                    filtered_df['log_' + molecule] = df[molecule]
                else:
                    # Fill with zeros those who are not present.
                    filtered_df['log_' + molecule] = 0
            
            record_dict = filtered_df.to_dict(orient="records")

            # Writing to TFRecord file
            #   The files follow the following name structure:
            #   {earth_type}_{origin_folder}_{original_range_of_samples}_{number_of_actual_samples}
            tfrecord_file = f'{earth_type}_{folder}_{original_parquet_range}_{len(record_dict)}.tfrecord'
            save_root = '/home/cleber/Documentos/Sarah/geexhp-main/TFRecord_Tests/TFRecord_data'
            if not os.path.exists(save_root):
                os.makedirs(save_root)
            save_path_file = os.path.join(save_root, tfrecord_file)

            with tf.io.TFRecordWriter(save_path_file) as writer:
                for sample in record_dict:
                    serialized_sample = serialize_sample(sample)
                    writer.write(serialized_sample)

            pbar.update(1)  # Update the progress bar for each file

🌍 Progress: |██████████████████████████████| 100% (72/72 files) ⏳ [00:35 elapsed]
